In [65]:
# X-Sec Framework


In [66]:
# init qwen model config
from dotenv import load_dotenv, find_dotenv
from langchain_community.chat_models import ChatTongyi

_ = load_dotenv(find_dotenv("./env/.env"))

llm_model = "qwen-max"

llm = ChatTongyi(temperature=0.8, model=llm_model)

In [67]:
# prepare ttp datas
#!python3.10 ./extract_groups.py
#!pip install tabulate
#!pip install pandas
#!pip install jinja2

import pandas as pd
from tabulate import tabulate
from mitreattack.stix20 import MitreAttackData

# example group id = intrusion-set--fd66436e-4d33-450e-ac4c-f7810f1c85f4
group_id = "intrusion-set--fd66436e-4d33-450e-ac4c-f7810f1c85f4"
selected_group_alias = "FIN13"
mitre_attack_data = MitreAttackData("./mitre-datas/enterprise-attack-15.1.json")
# Define techniques_df as an empty dataframe
techniques_df = pd.DataFrame()
# Get all techniques used by the group
techniques = mitre_attack_data.get_techniques_used_by_group(group_id)
# Check if there are any techniques for the group
if not techniques:
    raise ValueError(f"No techniques found for group {group_id}")
else:
    # Update techniques_df with the techniques
    techniques_df = pd.DataFrame(techniques)
# Create a copy of the DataFrame for generating the LLM prompt
# Add a 'Technique Name' column to both DataFrames
techniques_df["Technique Name"] = techniques_df["object"].apply(lambda x: x["name"])
# Add a 'ATT&CK ID' column to both DataFrames
techniques_df["ATT&CK ID"] = techniques_df["object"].apply(
    lambda x: mitre_attack_data.get_attack_id(x["id"])
)
# Add a 'Phase Name' column to both DataFrames
techniques_df["Phase Name"] = techniques_df["object"].apply(
    lambda x: x["kill_chain_phases"][0]["phase_name"]
)
# Drop duplicate techniques based on Phase Name, Technique Name and ATT&CK ID
techniques_df = techniques_df.drop_duplicates(
    ["Phase Name", "Technique Name", "ATT&CK ID"]
)

selected_techniques_df = (
    techniques_df.groupby("Phase Name", observed=False)[
        ["Phase Name", "Technique Name", "ATT&CK ID"]
    ]
    .apply(lambda x: x.sample(n=1) if len(x) > 0 else None)
    .reset_index(drop=True)
)

# print(
#    tabulate(
#        techniques_df[["Phase Name", "Technique Name", "ATT&CK ID"]],
#        headers=["Phase Name", "Technique Name", "ATT&CK ID"],
#        tablefmt="psql",
#    )
# )

# Create an empty list to hold the kill chain information
kill_chain = []
# Iterate over the rows in the DataFrame
for index, row in selected_techniques_df.iterrows():
    # Extract the phase and technique information
    phase_name = row['Phase Name']
    technique_name = row['Technique Name']
    attack_id = row['ATT&CK ID']
    # Add a string with this information to the kill chain list
    kill_chain.append(f"{phase_name}: {technique_name} ({attack_id})")
# Convert the kill chain list to a string with newline characters between each item
kill_chain_string = "\n".join(["- "+kc for kc in kill_chain])

print(kill_chain_string)

- collection: Data from Local System (T1005)
- command-and-control: Protocol Tunneling (T1572)
- credential-access: Security Account Manager (T1003.002)
- defense-evasion: Make and Impersonate Token (T1134.003)
- discovery: File and Directory Discovery (T1083)
- execution: Visual Basic (T1059.005)
- impact: Financial Theft (T1657)
- initial-access: Exploit Public-Facing Application (T1190)
- lateral-movement: SMB/Windows Admin Shares (T1021.002)
- persistence: Account Manipulation (T1098)
- reconnaissance: Network Topology (T1590.004)
- resource-development: Tool (T1588.002)


In [68]:
industry_list = [
    "Aerospace / Defense",
    "Agriculture / Food Services",
    "Automotive",
    "Construction",
    "Education",
    "Energy / Utilities",
    "Finance / Banking",
    "Government / Public Sector",
    "Healthcare",
    "Hospitality / Tourism",
    "Insurance",
    "Legal Services",
    "Manufacturing",
    "Media / Entertainment",
    "Non-profit",
    "Real Estate",
    "Retail / E-commerce",
    "Technology / IT",
    "Telecommunication",
    "Transportation / Logistics",
]

industry = "Technology / IT"

company_list = [
    "Small (1-50 employees)",
    "Medium (51-200 employees)",
    "Large (201-1,000 employees)",
    "Enterprise (1,001-10,000 employees)",
    "Large Enterprise (10,000+ employees)",
]
company_size = "Large Enterprise (10,000+ employees)"

cve_2024_5535 = """Issue summary: Calling the OpenSSL API function SSL_select_next_proto with an
empty supported client protocols buffer may cause a crash or memory contents to
be sent to the peer.
Impact summary: A buffer overread can have a range of potential consequences
such as unexpected application beahviour or a crash. In particular this issue
could result in up to 255 bytes of arbitrary private data from memory being sent
to the peer leading to a loss of confidentiality. However, only applications
that directly call the SSL_select_next_proto function with a 0 length list of
supported client protocols are affected by this issue. This would normally never
be a valid scenario and is typically not under attacker control but may occur by
accident in the case of a configuration or programming error in the calling
application.
The OpenSSL API function SSL_select_next_proto is typically used by TLS
applications that support ALPN (Application Layer Protocol Negotiation) or NPN
(Next Protocol Negotiation). NPN is older, was never standardised and
is deprecated in favour of ALPN. We believe that ALPN is significantly more
widely deployed than NPN. The SSL_select_next_proto function accepts a list of
protocols from the server and a list of protocols from the client and returns
the first protocol that appears in the server list that also appears in the
client list. In the case of no overlap between the two lists it returns the
first item in the client list. In either case it will signal whether an overlap
between the two lists was found. In the case where SSL_select_next_proto is
called with a zero length client list it fails to notice this condition and
returns the memory immediately following the client list pointer (and reports
that there was no overlap in the lists).
This function is typically called from a server side application callback for
ALPN or a client side application callback for NPN. In the case of ALPN the list
of protocols supplied by the client is guaranteed by libssl to never be zero in
length. The list of server protocols comes from the application and should never
normally be expected to be of zero length. In this case if the
SSL_select_next_proto function has been called as expected (with the list
supplied by the client passed in the client/client_len parameters), then the
application will not be vulnerable to this issue. If the application has
accidentally been configured with a zero length server list, and has
accidentally passed that zero length server list in the client/client_len
parameters, and has additionally failed to correctly handle a "no overlap"
response (which would normally result in a handshake failure in ALPN) then it
will be vulnerable to this problem.
In the case of NPN, the protocol permits the client to opportunistically select
a protocol when there is no overlap. OpenSSL returns the first client protocol
in the no overlap case in support of this. The list of client protocols comes
from the application and should never normally be expected to be of zero length.
However if the SSL_select_next_proto function is accidentally called with a
client_len of 0 then an invalid memory pointer will be returned instead. If the
application uses this output as the opportunistic protocol then the loss of
confidentiality will occur.
This issue has been assessed as Low severity because applications are most
likely to be vulnerable if they are using NPN instead of ALPN - but NPN is not
widely used. It also requires an application configuration or programming error.
Finally, this issue would not typically be under attacker control making active
exploitation unlikely.
The FIPS modules in 3.3, 3.2, 3.1 and 3.0 are not affected by this issue.
Due to the low severity of this issue we are not issuing new releases of
OpenSSL at this time. The fix will be included in the next releases when they
become available."""

In [69]:
# sequential chain
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate

## prompt for generating attack vector from MITRE MATRIX TTP model
with open("./ttp.md", "r") as f:
    ttp_template = f.read()

ttp_prompt_template = ChatPromptTemplate.from_template(ttp_template)
print(ttp_prompt_template.input_variables)

['company_size', 'industry', 'kill_chain_string', 'selected_group_alias']


In [70]:
# prompt for x-sec to analyze
with open("./x-sec.md", "r") as f:
    xsec_template = f.read()

xsec_prompt_template = ChatPromptTemplate.from_template(xsec_template)
print(xsec_prompt_template.input_variables)

['cve_2024_5535', 'scenario']


In [97]:
from langchain_core.output_parsers import StrOutputParser

# sequencial chain
# chain 1:
#   input=['company_size', 'industry', 'kill_chain_string', 'selected_group_alias']
#   output= ['scenario']
_ttp_chain = LLMChain(llm=llm, prompt=ttp_prompt_template, output_key="scenario")
ttp_chain = (ttp_prompt_template
    | llm
    | StrOutputParser()
)

# chain 2:
#   input= ['scenario']
#   output= summary
_xsec_chain = LLMChain(llm=llm, prompt=xsec_prompt_template, output_key="summary")
xsec_chain = xsec_prompt_template | llm | StrOutputParser()

In [98]:
# legacy sequentialchain invoke
from langchain.chains.sequential import SequentialChain

_overall_chain = SequentialChain(
   chains=[_ttp_chain, _xsec_chain],
   input_variables=[
       "company_size",
       "industry",
       "kill_chain_string",
       "selected_group_alias",
       "cve_2024_5535",
   ],
   output_variables=["scenario", "summary"],
   verbose=True,
)
response = _overall_chain(
    {
        "company_size": company_size,
        "industry": industry,
        "kill_chain_string": kill_chain_string,
        "selected_group_alias": selected_group_alias,
        "cve_2024_5535": cve_2024_5535,
    }
)
from IPython.display import display, Markdown

display(Markdown(response['summary']))



> Entering new SequentialChain chain...


In [96]:
from langchain.callbacks.tracers import ConsoleCallbackHandler

overall_chain = {
    "scenario": ttp_chain,
    "cve_2024_5535": lambda x: cve_2024_5535,
} | xsec_chain
response = overall_chain.invoke(
    {
        "company_size": company_size,
        "industry": industry,
        "kill_chain_string": kill_chain_string,
        "selected_group_alias": selected_group_alias,
        "cve_2024_5535": cve_2024_5535,
    },
    #config={"callbacks": [ConsoleCallbackHandler()]}, # configure for verbose logging
)

from IPython.display import display, Markdown
display(Markdown(response))

[chain/start] 

[chain:RunnableSequence] Entering Chain run with input:
{
  "company_size": "Large Enterprise (10,000+ employees)",
  "industry": "Technology / IT",
  "kill_chain_string": "- collection: Data from Local System (T1005)\n- command-and-control: Protocol Tunneling (T1572)\n- credential-access: Security Account Manager (T1003.002)\n- defense-evasion: Make and Impersonate Token (T1134.003)\n- discovery: File and Directory Discovery (T1083)\n- execution: Visual Basic (T1059.005)\n- impact: Financial Theft (T1657)\n- initial-access: Exploit Public-Facing Application (T1190)\n- lateral-movement: SMB/Windows Admin Shares (T1021.002)\n- persistence: Account Manipulation (T1098)\n- reconnaissance: Network Topology (T1590.004)\n- resource-development: Tool (T1588.002)",
  "selected_group_alias": "FIN13",
  "cve_2024_5535": "Issue summary: Calling the OpenSSL API function SSL_select_next_proto with an\nempty supported client protocols buffer may cause a crash or memory contents to\nbe sent to the pe

### Kill Chain of the Given Incident Scenario

The kill chain phases outlined in the scenario, based on the MITRE ATT&CK framework, are as follows:

1. **Reconnaissance**:
   - Technique T1590.004 (Network Topology Discovery): FIN13 gathers information about the target network's layout to identify points of entry.

2. **Initial Access**:
   - Technique T1190 (Exploit Public-Facing Application): The attackers exploit a vulnerability in a public-facing application to gain entry into the network.

3. **Execution**:
   - Technique T1059.005 (Visual Basic): Malware or scripts utilizing Visual Basic are executed once inside the network.

4. **Persistence**:
   - Technique T1098 (Account Manipulation): Attackers manipulate legitimate user accounts to maintain their presence in the system.

5. **Privilege Escalation**:
   - Technique T1134.003 (Make and Impersonate Token): FIN13 creates and impersonates tokens to gain higher-level privileges.

6. **Defense Evasion**:
   - Technique T1572 (Protocol Tunneling): Communications are tunneled to avoid detection.

7. **Credential Access**:
   - Technique T1003.002 (Security Account Manager): Attackers access the SAM database to harvest credentials.

8. **Discovery**:
   - Technique T1083 (File and Directory Discovery): FIN13 searches for valuable data and system configurations.

9. **Lateral Movement**:
   - Technique T1021.002 (SMB/Windows Admin Shares): The attackers use SMB shares to move laterally across the network.

10. **Collection**:
    - Technique T1005 (Data from Local System): Sensitive data is gathered from compromised systems.

11. **Exfiltration**:
    - Implied but not explicitly listed in the TTPs provided; typically involves data transfer out of the network, potentially via the established C2 channels.

12. **Impact**:
    - Technique T1657 (Financial Theft): The ultimate goal of stealing financial information or manipulating financial transactions.

### Open Source Software and Vulnerabilities

Considering the scenario's context and common technologies in a large enterprise IT infrastructure, some relevant open-source software could include:

1. **Apache HTTP Server**: Often used for public-facing applications, which could be targeted for initial access (T1190).
   - Example Vulnerability: CVE-2021-41773 (Critical) – Path Traversal leading to Remote Code Execution.

2. **OpenSSL**: For secure communication, which is relevant to command-and-control (T1572).
   - **CVE-2024-5535** (Low) – Described in your question; affects applications improperly using SSL_select_next_proto.

3. **Samba**: Involved in file sharing and could be relevant to lateral movement (T1021.002).
   - Example Vulnerability: CVE-2021-44142 (Critical) – Remote Code Execution via WDigest authentication.

### Assessment of CVE-2024-5535 in the Incident

Given the nature of CVE-2024-5535, which primarily affects applications that incorrectly use the SSL_select_next_proto function with an empty client protocols buffer, its direct impact on the described incident scenario seems limited. The scenario focuses on a targeted attack leveraging specific TTPs, such as exploiting public-facing applications and stealing financial data, rather than on vulnerabilities in cryptographic libraries like OpenSSL unless specifically exploited for initial access or command-and-control obfuscation.

However, if the attackers were to leverage SSL/TLS communications for their command-and-control channels and an application within the enterprise inadvertently exposed this vulnerability due to a programming error, there could be a minimal risk of information leakage (loss of confidentiality). Yet, given the low severity assessment and the specific conditions required for exploitation, it is not a central vulnerability in this scenario.

### Score (1-10) on the Possibility CVE-2024-5535 is Affected

Considering the scenario and the details of CVE-2024-5535, I would give it a score of **2**. The vulnerability's direct impact on the incident scenario is low due to its specific nature, requirement for application misconfiguration, and low severity rating. While it theoretically could contribute to an information leak if an affected service was part of the command-and-control infrastructure, it's not a primary concern compared to the other TTPs detailed in the scenario.